# Preparam el notebook

In [10]:
import os
import time
import itertools

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import image
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

from sklearn.preprocessing import MinMaxScaler
from skimage.feature import hog
from skimage import io, feature
from skimage.transform import rotate, resize
from skimage.util import random_noise
from scipy import ndimage

# Llegim totes les imatges del dataset

In [2]:
def get_image_files_by_folder(root_folder, image_extensions=('jpg', 'jpeg', 'png', 'gif', 'bmp')):
    image_files_by_folder = {}

    # Walk through the root folder and its subdirectories
    for folder_path, _, files in os.walk(root_folder):
        folder_name = os.path.basename(folder_path)
        image_files = []

        for file in files:
            file_extension = file.lower().split('.')[-1]
            if file_extension in image_extensions:
                image_files.append(os.path.join(folder_path, file))

        if image_files:
            image_files_by_folder[folder_name] = image_files

    return image_files_by_folder

In [3]:
train_dataset_path = "data/train"
images_to_load_train = get_image_files_by_folder(train_dataset_path)
images_to_load_train.keys()

dict_keys(['bedroom', 'coast', 'forest', 'highway', 'industrial', 'insidecity', 'kitchen', 'livingroom', 'mountain', 'office', 'opencountry', 'store', 'street', 'suburb', 'tallbuilding'])

In [4]:
test_dataset_path = "data/test"
images_to_load_test = get_image_files_by_folder(test_dataset_path)
images_to_load_test.keys()

dict_keys(['bedroom', 'coast', 'forest', 'highway', 'industrial', 'insidecity', 'kitchen', 'livingroom', 'mountain', 'office', 'opencountry', 'store', 'street', 'suburb', 'tallbuilding'])

# Cream el dataframe

In [11]:
def read_and_flatten_images(image_list, augment_data):
    image_data = []
    scaler = MinMaxScaler()
    
    start_time = time.time()
    for image_path in image_list:
        img = io.imread(image_path, as_gray=True)
        img = resize(img, (200, 200))
        if augment_data:
            flipped_img = img[:, ::-1]
            rotated_5 = rotate(img, 5)
            rotated_neg_5 = rotate(img, -5)
            blured = ndimage.uniform_filter(img)
            images = [img, flipped_img, rotated_5, rotated_neg_5, blured]
        else:
            images = [img]

        for image_to_process in images:
            hog_vectors = hog(image_to_process, orientations=9, pixels_per_cell=(20, 20),
                              cells_per_block=(10, 10), channel_axis=None, transform_sqrt=True)
            lbp = feature.local_binary_pattern(image_to_process, 24, 3, method='uniform')
            lbp_features = np.histogram(lbp, bins=np.arange(0, 24 + 3), range=(0, 24 + 2))[0]
            lbp_features_scaled = scaler.fit_transform(lbp_features.reshape(-1, 1))
            lbp_features_resized = scaler.inverse_transform(lbp_features_scaled)

            image_data.append(np.concatenate((hog_vectors, lbp_features_resized.squeeze())))
            #image_data.append(hog_vectors)
    
    print("Read + HOG + Data augmentation + LBP time:", time.time() - start_time, "seconds")
    return image_data

def create_dataframe_from_categories(images_to_load, augment_data=True):
    df = pd.DataFrame()
    for label, image_list in images_to_load.items():
        image_data = read_and_flatten_images(images_to_load[label], augment_data)
        next_part = pd.DataFrame(image_data)
        next_part['label'] = label
        df = pd.concat([df, next_part], ignore_index=True)
    return df

In [12]:
train_df = create_dataframe_from_categories(images_to_load_train)
test_df = create_dataframe_from_categories(images_to_load_test, augment_data=False)

Read + HOG + Data augmentation + LBP time: 9.81090784072876 seconds
Read + HOG + Data augmentation + LBP time: 9.754570960998535 seconds
Read + HOG + Data augmentation + LBP time: 9.904993295669556 seconds
Read + HOG + Data augmentation + LBP time: 9.715821743011475 seconds
Read + HOG + Data augmentation + LBP time: 9.85094428062439 seconds
Read + HOG + Data augmentation + LBP time: 9.799019813537598 seconds
Read + HOG + Data augmentation + LBP time: 9.813910484313965 seconds
Read + HOG + Data augmentation + LBP time: 9.810797929763794 seconds
Read + HOG + Data augmentation + LBP time: 9.844354391098022 seconds
Read + HOG + Data augmentation + LBP time: 9.71782398223877 seconds
Read + HOG + Data augmentation + LBP time: 9.718824625015259 seconds
Read + HOG + Data augmentation + LBP time: 9.850944519042969 seconds
Read + HOG + Data augmentation + LBP time: 9.873965501785278 seconds
Read + HOG + Data augmentation + LBP time: 9.825311183929443 seconds
Read + HOG + Data augmentation + LBP 

In [13]:
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,917,918,919,920,921,922,923,924,925,label
0,0.011354,0.004432,0.001141,0.003175,0.040967,0.090160,0.002581,0.004697,0.008418,0.012150,...,522.0,369.0,387.0,427.0,557.0,787.0,1173.0,4139.0,16613.0,bedroom
1,0.009562,0.018226,0.005070,0.002203,0.005281,0.002371,0.002201,0.003917,0.004864,0.029355,...,522.0,369.0,387.0,429.0,555.0,788.0,1172.0,4139.0,16613.0,bedroom
2,0.003996,0.000853,0.002064,0.022376,0.159685,0.063335,0.002938,0.004193,0.005589,0.004060,...,555.0,471.0,475.0,565.0,628.0,870.0,905.0,4821.0,13464.0,bedroom
3,0.203009,0.000000,0.000000,0.000078,0.000027,0.000163,0.000068,0.000055,0.000040,0.013108,...,500.0,463.0,492.0,527.0,636.0,824.0,898.0,5124.0,13515.0,bedroom
4,0.006962,0.003344,0.001867,0.004662,0.030570,0.121183,0.002288,0.001978,0.002625,0.004748,...,636.0,588.0,600.0,760.0,784.0,799.0,630.0,1742.0,10904.0,bedroom


In [14]:
train_df.shape

(7500, 927)

In [15]:
test_df.shape

(2985, 927)

# Cream els conjunts de test i d'entrenament

In [16]:
X_train = train_df.drop('label', axis=1)
y_train = train_df['label']

X_test = test_df.drop('label', axis=1)
y_test = test_df['label']

# Entrenam el model

In [17]:
svm = SVC(C=10, kernel='poly', random_state=33, degree=4, coef0=0.01)
start_time = time.time()
svm.fit(X_train, y_train)
print("fit time:", time.time() - start_time, "seconds")

fit time: 7.281625986099243 seconds


# Donam una predicció

In [18]:
y_predicted = svm.predict(X_test)

print(f"La precisició es de: {accuracy_score(y_test, y_predicted)}")

La precisició es de: 0.4525963149078727


# Quines categories tenen major error?

In [19]:
print("Classification report:\n", classification_report(y_test, y_predicted))

Classification report:
               precision    recall  f1-score   support

     bedroom       0.41      0.20      0.27       116
       coast       0.43      0.44      0.43       260
      forest       0.67      0.86      0.75       228
     highway       0.30      0.53      0.38       160
  industrial       0.38      0.39      0.38       211
  insidecity       0.43      0.62      0.51       208
     kitchen       0.34      0.43      0.38       110
  livingroom       0.40      0.28      0.33       189
    mountain       0.56      0.33      0.41       274
      office       0.46      0.72      0.56       115
 opencountry       0.46      0.39      0.42       310
       store       0.55      0.53      0.54       215
      street       0.46      0.45      0.45       192
      suburb       0.45      0.50      0.47       141
tallbuilding       0.40      0.23      0.29       256

    accuracy                           0.45      2985
   macro avg       0.45      0.46      0.44      2985
we